In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess
from IPython.display import Markdown, display


In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:6]}")
else:
    print("OpenRouter API Key not set (and this is optional)")



OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set (and this is optional)
Google API Key not set (and this is optional)
Grok API Key not set (and this is optional)
Groq API Key not set (and this is optional)
OpenRouter API Key exists and begins sk-or-


In [3]:
# Connect to client libraries

ollama_url = "http://localhost:11434/v1"
openrouter_url = "https://openrouter.ai/api/v1"

ollama = OpenAI(api_key="ollama", base_url=ollama_url)
openrouter = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)



In [4]:
models = ["openai/gpt-5.2", "anthropic/claude-sonnet-4.5", "x-ai/grok-4", "gemini-2.5-pro", "qwen3:4b", "deepseek-coder-v2", "gpt-oss:20b", "qwen/qwen3-coder-30b-a3b-instruct", "openai/gpt-oss-120b:free", ]

clients = {"openai/gpt-5.2": openrouter, "anthropic/claude-sonnet-4.5": openrouter, "x-ai/grok-4": openrouter, "gemini-2.5-pro": openrouter, "openai/gpt-oss-120b:free": openrouter, "qwen3:4b": ollama, "deepseek-coder-v2": ollama, "gpt-oss:20b": ollama, "qwen/qwen3-coder-30b-a3b-instruct": openrouter}


In [5]:
instruction = """
You are a code documentation assistant. Your sole task is to add clear, concise comments and docstrings to code provided by the user.

Instructions:
1. Greet the user with exactly: "Paste your code below and I'll add comments and docstrings to it."
2. When the user pastes code, automatically detect the programming language.
3. Return the EXACT same code — same logic, same structure, same variable names, same formatting — with only comments and docstrings added. Do NOT modify, refactor, optimize, or fix any part of the code.
4. Use the idiomatic comment and docstring style for the detected language:
   - Python: # for inline comments, \"\"\"triple-quote docstrings\"\"\" for functions/classes/modules
   - JavaScript/TypeScript: // for inline comments, /** JSDoc */ for functions/classes
   - Go: // for inline comments, // preceding function signature for godoc
   - Java/Kotlin: // for inline comments, /** Javadoc */ for classes/methods
   - Rust: // for inline comments, /// doc comments for functions/structs
   - C/C++: // for inline comments, /** Doxygen */ for functions/structs
   - Ruby: # for inline comments, yard-style comments for methods/classes
   - For any other language, follow its established documentation conventions.
5. Comment coverage:
   - Add a top-level docstring/comment summarizing the file or snippet's purpose.
   - Add docstrings to every function, method, and class describing what it does, its parameters, and its return value.
   - Add inline comments only where the logic is non-obvious — do NOT narrate every line.
6. Output ONLY the commented code inside a single fenced code block tagged with the detected language. No explanations, no preamble, no follow-up text outside the code block.
"""

In [ ]:
import re

def comment_code(model, code):
    client = clients[model]
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": instruction},
            {"role": "user", "content": code}
        ]
    )
    reply = response.choices[0].message.content
    reply = re.sub(r"^```\w*\n", "", reply)
    reply = re.sub(r"\n```$", "", reply)
    return reply

In [ ]:
from styles import CSS

with gr.Blocks(css=CSS, theme=gr.themes.Monochrome(), title="Code Commenter") as ui:
    gr.Markdown("## Paste your code below and I'll add comments and docstrings to it.")
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            code_input = gr.Code(
                label="Your code",
                value="",
                lines=26
            )
        with gr.Column(scale=6):
            code_output = gr.Code(
                label="Commented code",
                value="",
                lines=26
            )

    with gr.Row(elem_classes=["controls"]):
        model = gr.Dropdown(models, value=models[0], show_label=False)
        comment_btn = gr.Button("Add Comments", elem_classes=["convert-btn"])

    comment_btn.click(fn=comment_code, inputs=[model, code_input], outputs=[code_output])

ui.launch(inbrowser=True)
